In [21]:
# Importação dos dados
"""
Geral
1. Quantos restaurantes únicos estão registrados?
2. Quantos países únicos estão registrados?
3. Quantas cidades únicas estão registradas?
4. Qual o total de avaliações feitas?
5. Qual o total de tipos de culinária registrados?
"""

import pandas as pd
import sys
import numpy
import inflection
import plotly.express as px
# Funções:


def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"
    
    
COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]

def rename_columns(dataframe):
    df = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df


# Extração dos dados e limpeza dos dados
### Extração
df_raw = pd.read_csv("zomato.csv")
df = df_raw.copy()

### Limpeza dos dados
df.columns
df = rename_columns(df)
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}


# Mudança de coluna
df['country_code'] = df['country_code'].map(COUNTRIES)

# Criação de uma nova coluna cost range
df["cost_range"] = df.loc[:, "price_range"].apply(lambda x : create_price_tye(x))
df["cuisines"] = df["cuisines"].astype(str)
df["cuisines"] = df.loc[:, "cuisines"].apply(lambda x: x.split(",")[0])
df.columns
df.head()
df.columns
df["currency"].unique()

# PENSAR DEPOIS NISSO AQUI
# def dolar_change(currency, average_cost_for_two):
#     if currency == "Botswana Pula(P)":
#         dolar = average_cost_for_two * 0.74    
#     return dolar

#df["average_cost_for_two_in_dolar"] = df["average_cost_for_two"].apply(lambda x: dolar_change(x, ))

# TOMEI A INICIATIVA DE TRANSFORMAR TRANSFORMAR TUDO EM DOLAR



# Transformar os obejtos em strings. Exemplo: A coluna cuisines
df.head()

df = df.astype({"restaurant_id": int})
# Depois a gente volta aqui

# Visão cidades

### Top 10 cidades com mais restaurantes na base de dados

In [ ]:
df_aux1 = df.loc[:, ["city", "restaurant_id", "country_code"]].groupby(["city", "country_code"]).nunique().reset_index()
df_aux1 = df.loc[:, ["city", "restaurant_id", "country_code"]].groupby(["city", "country_code"]).nunique().reset_index()
df_aux2 = df_aux1.head(10)
fig = px.bar(df_aux2, 
            x="city", 
            y ="restaurant_id", 
            color="country_code", 
            labels={"restaurant_id": "Número de restaurantes", "city" : "Cidades"})
fig.show()

### Top 10 cidades com  restaurantes e médias de avaliação acima de 4

In [36]:
linhas_selecionadas = df["aggregate_rating"] > 4
df_aux = df.loc[linhas_selecionadas, ["city", "aggregate_rating", "country_code"]].groupby(["city", "country_code"]).mean().reset_index()
df_aux = df_aux.sort_values(by = "aggregate_rating", ascending=False).reset_index(drop=True)
df_aux1 = df_aux.head(10)
fig = px.bar(df_aux1, 
            x="city", 
            y ="aggregate_rating", 
            color="country_code", 
            labels={"restaurant_id": "Número de restaurantes", "city" : "Cidades"})
fig.show()

In [65]:
df.dtypes

restaurant_id             int64
restaurant_name          object
country_code             object
city                     object
address                  object
locality                 object
locality_verbose         object
longitude               float64
latitude                float64
cuisines                 object
average_cost_for_two      int64
currency                 object
has_table_booking         int64
has_online_delivery       int64
is_delivering_now         int64
switch_to_order_menu      int64
price_range               int64
aggregate_rating        float64
rating_color             object
rating_text              object
votes                     int64
cost_range               object
dtype: object

### Top 5 cidades com  restaurantes e médias de avaliação abaixo de 2.5


In [68]:
linhas_selecionadas = (df["aggregate_rating"] == 0)
df_aux = df.loc[linhas_selecionadas, ["city", "aggregate_rating"]].groupby(["city"]).count().reset_index()
df_aux = df_aux.sort_values(by = "aggregate_rating", ascending=False).reset_index(drop=True)
df_aux

,city,aggregate_rating
0,Gangtok,47
1,Ooty,22
2,São Paulo,17
3,Brasília,15
4,Rio de Janeiro,11
5,Manchester,7
6,Clarens,4
7,Shimla,4
8,Edinburgh,2


In [ ]:
# Cidades notas zeradas
linhas_selecionadas = df["aggregate_rating"] <2.5
df_aux1 = df.loc[linhas_selecionadas, ["city", "aggregate_rating"]].groupby(["city"]).mean().reset_index()
df_aux1 = df_aux1.sort_values(by = "aggregate_rating", ascending=True).reset_index(drop=True)
df_aux2 = df_aux1.loc[df_aux1["aggregate_rating"] == 0, :]
df_aux2
# lst = df_aux2['city'].unique()
# for i in lst:
#     print(i)